In [1]:
# Copyright 2021 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# =====

# Training and Deploying Multi-Stage Recommender Systems

Industrial recommender systems are made up of complex pipelines requiring multiple steps including feature engineering and
preprocessing, a retrieval model for candidate generation, filtering, a feature store query, a ranking model for scoring, and an ordering
stage. These pipelines need to be carefully deployed as a set, requiring coordination during their development and deployment. Data
scientists, ML engineers, and researchers might focus on different stages of recommender systems, however they share a common
desire to reduce the time and effort searching for and combining boilerplate code coming from different sources or writing custom
code from scratch to create their own RecSys pipelines.

This tutorial introduces the Merlin framework which aims to make the development and deployment of recommender systems
easier, providing methods for evaluating existing approaches, developing new ideas and deploying them to production. There are
many techniques, such as different model architectures (e.g. MF, DLRM, DCN, etc), negative sampling strategies, loss functions or
prediction tasks (binary, multi-class, multi-task) that are commonly used in these pipelines. Merlin provides building blocks that allow
RecSys practitioners to focus on the “what” question in designing their model pipeline instead of “how”. Supporting research into new
ideas within the RecSys spaces is equally important and Merlin supports the addition of custom components and the extension of
existing ones to address gaps.

In this tutorial, participants will learn: 
   - (i) how to easily implement common recommender system techniques for comparison, 
   - (ii) how to modify components to evaluate new ideas,
   - (iii) deploying recommender systems, bringing new ideas to production- using an open source framework Merlin and its libraries.

## 1. Implementing popular RecSys architectures and algorithms with Merlin Models

**Learning Objectives of this lab**

- Introduction to the open source framework Merlin and its libraries- NVTabular and Merlin Models
- Pre-processing and feature engineering with NVTabular
- Build and train common recommender systems models with Merlin Models

### NVIDIA Merlin

Merlin is an open-source framework for building large-scale (deep learning) recommender systems. It is designed to support recommender systems end-to-end from ETL to training to deployment on CPU or GPU. Common deep learning frameworks are integrated such as TensorFlow or PyTorch. Its key benefits are the easy-to-use APIs, accelerations with GPU and scaling to multi-GPU or multi-node systems.

![Merlin](./images/Merlin.png)

### Merlin Models

[Merlin Models](https://github.com/NVIDIA-Merlin/models) is a library to make it easy for users in industry or academia to train and deploy recommender models with best practices baked into the library. This will let users in industry easily train standard models against their own dataset, getting high performance GPU accelerated models into production. This will also let researchers to build custom models by incorporating standard components of deep learning recommender models, and then benchmark their new models on example offline datasets.Core features are:
- Unified API enables users to create models in TensorFlow or PyTorch
- Deep integration with NVTabular for ETL and model serving
- Flexible APIs targeted to both production and research
- Many different recommender system architectures (tabular, two-tower, sequential) or tasks (binary, multi-class classification, multi-task)

### NVTabular 

[NVTabular](https://github.com/NVIDIA-Merlin/NVTabular) is a feature engineering and preprocessing library for tabular data that is designed to easily manipulate terabyte scale datasets and train deep learning (DL) based recommender systems. It provides high-level abstraction to simplify code and accelerates computation on the GPU using the RAPIDS Dask-cuDF library. NVTabular helps data scientists and ML engineers to:
- process datasets that exceed GPU and CPU memory without having to worry about scale
- focus on what to do with the data and not how to do it by using abstraction at the operation level
- prepare datasets quickly and easily for experimentation so that more models can be trained.

![Merlin](./images/schema.png)

That's a short introduction into Merlin, NVTabular and Merlin Models. If you are interested to learn more, we provide many examples in our GitHub repositories. 

Let's get started!

### Feature Engineering on GPU with NVTabular

In this notebook, we use publicly available [eCommerce](https://www.kaggle.com/mkechinov/ecommerce-behavior-data-from-multi-category-store) behavior dataset. The full dataset contains 7 months of data (from October 2019 to April 2020) from a large multi-category online store. Each row in the file represents an event. All events are related to products and users. Each event is like many-to-many relation between products and users. Data collected by Open CDP project and the source of the dataset is [REES46 Marketing Platform](https://rees46.com/). You can visit [Kaggle](https://www.kaggle.com/mkechinov/ecommerce-behavior-data-from-multi-category-store) to download the csv files. 

The csv files from 2019-Oct to 2020-April have been pre-processed in the `01_Introduction` notebook and we created train and valiation data sets to train and validate our models.

In the feature engineering step, we do following data operations:

- Categorify Categories
- Create temporal features
- Transform Continuous features
- Count Encoding (JoinGroupBy)
- Target Encoding

**Import Required Libraries**

In [1]:
import os

import glob
import cudf 
import pandas as pd
import numpy as np
import nvtabular as nvt
from nvtabular.ops import *
import gc

from merlin.schema.tags import Tags
import merlin.models.tf as mm
from merlin.io.dataset import Dataset

import tensorflow as tf

2022-08-11 20:41:08.144752: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-11 20:41:09.295040: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 16249 MB memory:  -> device: 0, name: Quadro GV100, pci bus id: 0000:2d:00.0, compute capability: 7.0


In [2]:
data_path = './'
output_path = os.path.join(data_path,'processed_nvt')

In [4]:
train_dataset = nvt.Dataset(os.path.join(data_path, 'train.parquet'))
valid_dataset = nvt.Dataset(os.path.join(data_path, 'valid.parquet'))

/usr/local/lib/python3.8/dist-packages/cudf/core/frame.py:384: UserWarning: The deep parameter is ignored and is only included for pandas compatibility.
  warnings.warn(


In [8]:
# Relative price to the average price for the product_id
def relative_price_to_avg_categ(col, gdf):
    epsilon = 1e-5
    col = ((gdf['price'] - col) / (col + epsilon)) * (col > 0).astype(int)
    return col

In [9]:
user_id = ["user_id"] >> Categorify() >> TagAsUserID()

weekday = (
    ["timestamp"] >> 
    LambdaOp(lambda col: col.dt.weekday) >> 
    Rename(name ='ts_weekday')
)

day = (
    ["timestamp"] >> 
    LambdaOp(lambda col: col.dt.day) >> 
    Rename(name ='ts_day')
)

hour = (
    ["timestamp"] >> 
    LambdaOp(lambda col: col.dt.hour) >> 
    Rename(name ='ts_hour')
)

context_features = (
    (hour + weekday + day) 
    >> Categorify() >> TagAsUserFeatures()
)

# count encode `userId`
count_logop_feat = (
    ["product_id"]
    >> JoinGroupby(cont_cols=["target"], stats=["count"], out_path='./categories_count')
    >> LogOp()
    >> TagAsUserFeatures()
)


item_id = ["product_id"] >> Categorify() >> TagAsItemID()

item_features = ["cat_0", "cat_1", "cat_2", "brand"] >> Categorify() >> TagAsItemFeatures()

# Target encode cat columns
cat_groups =  nvt.ColumnSelector(['user_id', 'brand', 'cat_1', 'cat_2'])
label = nvt.ColumnSelector(["target"])
te_features = cat_groups >> TargetEncoding(label)
te_features_norm = te_features >> Normalize() >> TagAsItemFeatures()

price = (
    ['price']
    >> FillMissing(0)
    >> LogOp()
    >> Normalize()
    >> TagAsItemFeatures()
)   

avg_category_id_pr = ['product_id'] >> JoinGroupby(cont_cols =['price'], stats=["mean"]) >> Rename(name='avg_category_id_price')

relative_price_to_avg_category = (
    avg_category_id_pr 
    >> LambdaOp(relative_price_to_avg_categ, dependency=['price']) 
    >> Rename(name='relative_price')
    >> AddMetadata(tags=["item", Tags.CONTINUOUS])
)


target = (
    ["target"] 
    >> nvt.ops.AddMetadata(tags=[Tags.BINARY_CLASSIFICATION, "target"]) 
)

outputs = user_id  + context_features + item_id + item_features + price + relative_price_to_avg_category + count_logop_feat+ te_features_norm + target

workflow = nvt.Workflow(outputs)

In [10]:
%%time
workflow.fit(train_dataset)

workflow.transform(train_dataset).to_parquet(
    output_path=os.path.join(output_path, "train")
)

workflow.transform(valid_dataset).to_parquet(
    output_path=os.path.join(output_path, "valid")
)

/usr/local/lib/python3.8/dist-packages/cudf/core/frame.py:384: UserWarning: The deep parameter is ignored and is only included for pandas compatibility.
  warnings.warn(


CPU times: user 4.16 s, sys: 3.86 s, total: 8.02 s
Wall time: 8.51 s


In [5]:
tmp = pd.read_parquet('./processed_nvt/train/part_0.parquet')

In [6]:
tmp.head(2)

,user_id,ts_hour,ts_weekday,ts_day,product_id,cat_0,cat_1,cat_2,brand,price,relative_price,product_id_count,TE_user_id_target,TE_brand_target,TE_cat_1_target,TE_cat_2_target,target
0,1587740,24,2,18,1,1,1,1,1,0.3723,0.022421,12.505351,-0.228408,0.516618,0.616164,0.992369,0
1,1587740,24,2,18,1,1,1,1,1,0.3723,0.022421,12.505351,-0.228408,0.518844,0.616738,0.992705,0


NVTabular exported the schema file of our processed dataset. The schema.pbtxt is a protobuf text file contains features metadata, including statistics about features such as cardinality, min and max values and also tags based on their characteristics and dtypes (e.g., categorical, continuous, list, item_id). The metadata information is loaded from schema and their tags are used to automatically set the parameters of Merlin Models. In other words, Merlin Models relies on the schema object to automatically build all necessary input and output layers.

In [3]:
train = Dataset(os.path.join(output_path, "train", "*.parquet"), part_size="500MB")
valid = Dataset(os.path.join(output_path, "valid", "*.parquet"), part_size="500MB")

# define schema object
schema = train.schema

/usr/local/lib/python3.8/dist-packages/cudf/core/frame.py:384: UserWarning: The deep parameter is ignored and is only included for pandas compatibility.
  warnings.warn(


In [4]:
#schema

In [5]:
target_column = schema.select_by_tag(Tags.TARGET).column_names[0]
target_column

'target'

### DLRM Model

Deep Learning Recommendation Model (DLRM) architecture is a popular neural network model originally proposed by Facebook in 2019 as a personalization deep learning model.

![DLRM](./images/DLRM.png)

DLRM accepts two types of features: categorical and numerical.

- For each categorical feature, an embedding table is used to provide dense representation to each unique value.
- For numerical features, they are fed to model as dense features, and then transformed by a simple neural network referred to as "bottom MLP". This part of the network consists of a series of linear layers with ReLU activations.
- The output of the bottom MLP and the embedding vectors are then fed into the dot product interaction operation (see Pairwise interaction step). The output of "dot interaction" is then concatenated with the features resulting from the bottom MLP (we apply a skip-connection there) and fed into the "top MLP" which is also a series of dense layers with activations ((a fully connected NN).
- The model outputs a single number (here we use sigmoid function to generate probabilities) which can be interpreted as a likelihood of a certain user clicking on an ad, watching a movie, or viewing a news page.

In [17]:
model = mm.DLRMModel(
    schema,
    embedding_dim=64,
    bottom_block=mm.MLPBlock([128, 64]),
    top_block=mm.MLPBlock([128, 64, 32]),
    prediction_tasks=mm.BinaryClassificationTask(target_column),
)

In [18]:
model.compile(optimizer='adam', run_eagerly=False, metrics=[tf.keras.metrics.AUC()])
model.fit(train, validation_data=valid, batch_size=4096, epochs=2)

Epoch 1/2
2443/2443 [==============================] - 146s 48ms/step - loss: 0.4777 - auc: 0.8309 - regularization_loss: 0.0000e+00 - val_loss: 0.6524 - val_auc: 0.7087 - val_regularization_loss: 0.0000e+00
Epoch 2/2
2443/2443 [==============================] - 116s 47ms/step - loss: 0.3697 - auc: 0.9058 - regularization_loss: 0.0000e+00 - val_loss: 0.7171 - val_auc: 0.6961 - val_regularization_loss: 0.0000e+00


### DCN Model

DCN-V2 is an architecture proposed as an improvement upon the original [DCN model](https://arxiv.org/pdf/1708.05123.pdf). The explicit feature interactions of the inputs are learned through cross layers, and then combined with a deep network to learn complementary implicit interactions. The overall model architecture is depicted in Figure below, with two ways to combine the cross network with the deep network: (1) stacked and (2) parallel. The output of the embbedding layer is the concatenation of all the embedded vectors and the normalized dense features: x<sub>0</sub> = [x<sub>embed,1</sub>; . . . ; x<sub>embed,𝑛</sub>; 𝑥<sub>dense</sub>].

![DCN](./images/DCN.png)

<a href="https://arxiv.org/abs/2008.13535">Image Source: DCN V2 paper</a>

In this example, we build `DCN-v2 stacked` architecture. 

In [19]:
model = mm.DCNModel(
    schema,
    depth=2,
    deep_block=mm.MLPBlock([64, 32]),
    prediction_tasks=mm.BinaryClassificationTask(target_column),
)

In [20]:
model.compile('adam', run_eagerly=False, metrics=[tf.keras.metrics.AUC()])
model.fit(train, validation_data=valid, batch_size=4096, epochs=2)

Epoch 1/2
2443/2443 [==============================] - 125s 46ms/step - loss: 0.4443 - auc_1: 0.8580 - regularization_loss: 0.0000e+00 - val_loss: 0.6605 - val_auc_1: 0.6883 - val_regularization_loss: 0.0000e+00
Epoch 2/2
2443/2443 [==============================] - 111s 45ms/step - loss: 0.3048 - auc_1: 0.9375 - regularization_loss: 0.0000e+00 - val_loss: 0.7333 - val_auc_1: 0.6498 - val_regularization_loss: 0.0000e+00


In [21]:
model.evaluate(valid, batch_size=1024, return_dict=True)

1023/1023 [==============================] - 10s 9ms/step - loss: 0.7333 - auc_1: 0.6498 - regularization_loss: 0.0000e+00


{'loss': 0.7332543134689331,
 'auc_1': 0.6497794985771179,
 'regularization_loss': 0.0}

In [22]:
model = mm.DCNModel(
    schema,
    depth=2,
    deep_block=mm.MLPBlock([64, 32]),
    prediction_tasks=mm.BinaryClassificationTask(target_column),
)

model.compile('adam', run_eagerly=False, metrics=[tf.keras.metrics.AUC()])
model.fit(train, validation_data=valid, batch_size=4096, epochs=2, class_weight = {0: 1, 1: 2})

Epoch 1/2
2443/2443 [==============================] - 124s 47ms/step - loss: 0.6293 - auc_2: 0.8586 - regularization_loss: 0.0000e+00 - val_loss: 0.6809 - val_auc_2: 0.6775 - val_regularization_loss: 0.0000e+00
Epoch 2/2
2443/2443 [==============================] - 112s 46ms/step - loss: 0.4177 - auc_2: 0.9403 - regularization_loss: 0.0000e+00 - val_loss: 0.8116 - val_auc_2: 0.6333 - val_regularization_loss: 0.0000e+00


**Improve DCN Model performance**

Hyperparameter tuning (optimization) is an important phenomenon to find the possible best sets of hyperparameters to build and train the model from a given dataset. Hyperparameter tuning can be done manually or be managed by an algorithm like grid search and bayesian optimization. The latter optimizes the search for the best hyperparameters guided by a metric that needs to be maximized or minimized.

Below, we  showcase how we can inject certain hyperparameters to our DCN model and set their values to improve the model performance metrics, which is AUC for us.

In [6]:
lr_decay_rate = 0.93
optimizer = "adam"

def get_optimizer():
    lerning_rate =  0.008
    if lr_decay_rate:
        lerning_rate = tf.keras.optimizers.schedules.ExponentialDecay(
            0.008,
            decay_steps=100,
            decay_rate=0.93,
            staircase=True,
        )

    if optimizer == "adam":
        opt = tf.keras.optimizers.Adam(
            learning_rate=lerning_rate,
        )

    return opt

opt = get_optimizer()

In [8]:
from merlin.models.utils import schema_utils
embedding_dims = {}

item_id_feature_name = schema.select_by_tag(Tags.ITEM_ID).column_names[0]
item_id_domain = schema_utils.categorical_domains(schema)[
    item_id_feature_name
]
embedding_dims[item_id_domain] = 128

user_id_feature_name = schema.select_by_tag(Tags.USER_ID).column_names[0]
user_id_domain = schema_utils.categorical_domains(schema)[
    user_id_feature_name
]
embedding_dims[user_id_domain] = 128

embedding_options = mm.EmbeddingOptions(
    embedding_dims=embedding_dims,
    infer_embedding_sizes=True,
    embeddings_l2_reg=1e-5
    )

In [9]:
model = mm.DCNModel(
    schema,
    depth=2,
    deep_block=mm.MLPBlock(
        [64, 32],
        activation='relu',
        no_activation_last_layer=False,
        dropout=0.015,
    ),
    stacked=True,
    embedding_options=embedding_options,
    prediction_tasks=mm.BinaryClassificationTask('target'),
)

In [10]:
model.compile(optimizer=opt, run_eagerly=False, metrics=[tf.keras.metrics.AUC()])
model.fit(train, validation_data=valid, batch_size=4096, epochs=2, class_weight = {0: 1, 1: 2})

Epoch 1/2
2443/2443 [==============================] - 193s 67ms/step - loss: 0.6178 - auc: 0.8712 - regularization_loss: 0.0133 - val_loss: 0.7135 - val_auc: 0.6844 - val_regularization_loss: 0.0078
Epoch 2/2
2443/2443 [==============================] - 164s 67ms/step - loss: 0.3207 - auc: 0.9686 - regularization_loss: 0.0172 - val_loss: 0.9698 - val_auc: 0.6374 - val_regularization_loss: 0.0078


### XGBoost

[XGBoost](https://xgboost.ai/), which stands for Extreme Gradient Boosting, is a scalable, distributed gradient-boosted decision tree (GBDT) machine learning library. It provides parallel tree boosting and is the leading machine learning library for regression, classification, and ranking problems.

A Gradient Boosting Decision Trees (GBDT) is a decision tree ensemble learning algorithm similar to random forest, for classification and regression. Ensemble learning algorithms combine multiple machine learning algorithms to obtain a better model. Both random forest and GBDT build a model consisting of multiple decision trees. The difference is in how the trees are built and combined.

The term “gradient boosting” comes from the idea of “boosting” or improving a single weak model by combining it with a number of other weak models in order to generate a collectively strong model. Gradient boosting is an extension of boosting where the process of additively generating weak models is formalized as a gradient descent algorithm over an objective function. Gradient boosting sets targeted outcomes for the next model in an effort to minimize errors. Targeted outcomes for each case are based on the gradient of the error (hence the name gradient boosting) with respect to the prediction.

XGBoost is a scalable and highly accurate implementation of gradient boosting that pushes the limits of computing power for boosted tree algorithms, being built largely for energizing machine learning model performance and computational speed. With XGBoost, trees are built in parallel, instead of sequentially like GBDT. You can read more about XGBoost [here](https://www.nvidia.com/en-us/glossary/data-science/xgboost/).

In [7]:
# from merlin.core.utils import Distributed
from merlin.models.xgb import XGBoost

In order to facilitate training on data larger than the available GPU memory, the training will leverage Dask. All the complexity of starting a local dask cluster is hidden in the Distributed context manager.

Without further ado, let's train.

In [6]:
xgb_booster_params = {
    'objective':'binary:logistic',
    'tree_method':'gpu_hist',
    'eval_metric': "auc"
}

xgb_train_params = {
    'num_boost_round': 100,
    'verbose_eval': 20,
    'early_stopping_rounds': 100,
}


with Distributed():
    model = XGBoost(schema=schema, **xgb_booster_params)
    model.fit(
        train,
        evals=[(valid, 'validation_set'),],
        **xgb_train_params
    )
    metrics = model.evaluate(valid)

In [9]:
metrics

{'auc': 0.6393621079818755}